In [1]:
import numpy as np

# Soft NMS Implementation

In [2]:
# iou function for implementing Soft-NMS
def iou(box1, box2):
    [box1_x1, box1_y1, box1_x2, box1_y2] = box1
    [box2_x1, box2_y1, box2_x2, box2_y2] = box2
    xi1 = max(box1_x1, box2_x1)
    yi1 = max(box1_y1, box2_y1)
    xi2 = min(box1_x2, box2_x2)
    yi2 = min(box1_y2, box2_y2)
    inter_width = max(0, yi2 - yi1)
    inter_height = max(0, xi2 - xi1)
    inter_area = inter_width * inter_height
    box1_area = (box1_x2 - box1_x1) * ((box1_y2 - box1_y1))
    box2_area = (box2_x2 - box2_x1) * ((box2_y2 - box2_y1))
    union_area = box1_area + box2_area - inter_area
    iou = inter_area / union_area
    return iou

#### Implementation of soft NMS algorithm

In [3]:
def soft_nms(scores: np.array, boxes: np.array,sigma=0.5):
    # scores - shape (,n) numpy array that contains scores of bounding boxes
    # boxes - shape (n,4) numpy array that contains bounding boxes information.
    #        Each element in array is a size=4 array of [x1,y1,x2,y2]
    #        Where x1,y1 are the bottom left coordinates of bounding boxes,
    #        and x2,y2 are the top right coordinates of bounding boxes.
    # START of your implementation
    N = len(boxes)
    pos = 0
    
    for i in range(N):
        maxscore = scores[i]
        maxpos = i
        pos = i + 1
        # get max box
        while pos < N:
            if maxscore < scores[pos]:
                maxscore = scores[pos]
                maxpos = pos
            pos = pos + 1
        
        # swap box wih max score and current box
        tx1 = boxes[maxpos,0]
        ty1 = boxes[maxpos,1]
        tx2 = boxes[maxpos,2]
        ty2 = boxes[maxpos,3]
        boxes[maxpos]= boxes[i]
        boxes[i] = [tx1,ty1,tx2,ty2]
        scores[i], scores[maxpos] = scores[maxpos], scores[i]
        
        pos = i + 1
        
        # recalculate scores for boxes thatt intersect with the box with max score
        while pos < N:
            #current box
            box1 = boxes[pos]
            score1 = scores[pos]
            
            #box with max score
            box2 = boxes[i]
            score2 = scores[i]
            
            # calculate iou
            iou_val = iou(box1, box2)
            
            # score rescaling
            if iou_val > 0:
                weight = np.exp(-(iou_val * iou_val)/sigma)
                scores[pos] = weight*scores[pos]
            pos = pos + 1
    
    # save the new final values
    new_bbox = boxes
    new_scores = scores

    # END of your implementation
    # new_bbox - shape (n,4) numpy array that contains new bounding boxes information
    # new_scores - shape (n,) numpy array that contains new bounding boxes scores
    return new_bbox, new_scores

In [4]:
boxes = np.array([[200, 200, 400, 400], [220, 220, 420, 420], [200, 240, 400, 440], [240, 200, 440, 400], [1, 1, 2, 2]], dtype=np.float32)
scores = np.array([0.9, 0.8, 0.7, 0.6, 0.5], dtype=np.float32)

boxes, scores = soft_nms(scores, boxes, sigma=0.5)
print("scores[2] = " + str(scores[2]))
print("boxes[2] = " + str(boxes[2]))
print("scores = " + str(scores))
print("boxes = " + str(boxes))
print("scores.shape = " + str(scores.shape))
print("boxes.shape = " + str(boxes.shape))

assert scores.shape == (5,), "Wrong shape"
assert boxes.shape == (5, 4), "Wrong shape"

assert np.isclose(scores[2], 0.31670862), "Wrong value on scores[2]"
assert np.allclose(boxes[2], [220,220,420,420]), "Wrong value on boxes[2]"

assert np.allclose(scores, np.array([0.9,0.5,0.31670862,0.11392745,0.06270898])), "Wrong value on scores"
assert np.allclose(boxes, np.array([[200, 200, 400, 400], [1, 1, 2, 2], [220, 220, 420, 420], [200, 240, 400, 440], [240, 200, 440, 400]])), "Wrong value on boxes"

print("\033[92m All tests passed!")

scores[2] = 0.31670862
boxes[2] = [220. 220. 420. 420.]
scores = [0.9        0.5        0.31670862 0.11392745 0.06270898]
boxes = [[200. 200. 400. 400.]
 [  1.   1.   2.   2.]
 [220. 220. 420. 420.]
 [200. 240. 400. 440.]
 [240. 200. 440. 400.]]
scores.shape = (5,)
boxes.shape = (5, 4)
 All tests passed!
